<a href="https://colab.research.google.com/github/rafs-santos/Learn_Methods/blob/main/manim_convolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt update
!sudo apt install libcairo2-dev ffmpeg \
    texlive texlive-latex-extra texlive-fonts-extra \
    texlive-latex-recommended texlive-science \
    tipa libpango1.0-dev
!pip install manim
!pip install IPython --upgrade

In [3]:
from manim import *
import scipy.integrate
import numpy as np

Manim Community v0.17.3

# Convolução

Nessa seção será animado a convolução entre dois sinais "contínuos", sendo esses sinais definidos como

$$x(t)=\left\{
\begin{array}{lr}
        0, & t< -0.5\\
        1, &  -0.5\leq t < 0.5\\
        0, & t\geq 0.5
        \end{array}\right.
$$

Esse sinal foi escolhido de forma a animação corresponder a animação feita na página da wikipedia, conforme link abaixo.

https://pt.wikipedia.org/wiki/Convolu%C3%A7%C3%A3o

link da animação:

https://pt.wikipedia.org/wiki/Ficheiro:Convolution_of_box_signal_with_itself2.gif


A integral de convolução pode ser definida como
$$
(f*g)(t)=h(t)=\int_{-∞}^{∞}f(\tau)\cdot g(t-\tau)d\tau
$$

Antes da animação da convolução, faz-se um esforço para entender o processo de realização da conta. A animação abaixo mostra a inversão no tempo e o deslocamento da função.

Para melhor visualização modifica-se a equação que gera o sinal, sendo definido em $[0,~1]$ e não no intervalo $[-0.5,~0.5]$.

In [52]:
%%manim -qh -v WARNING SlideFunc

# define the function in parts
def func0(tmp):
  x = np.zeros_like(tmp)
  # assuming tmp is an array
  for i in range(len(tmp)):
    if(tmp[i] < 0 ):
      x[i] = 0
    elif(tmp[i] >= 0 and tmp[i] < 1):
      x[i] = 1
    else:
      x[i] = 0
  return x

class SlideFunc(Scene):
  def construct(self):
    self.camera.background_color = "#FFFFFF"

    # frequência de amostragem e vetor de tempo
    freq_sample = 500
    t_final = 5
    tmp = np.arange(-t_final*freq_sample, t_final*freq_sample+1)*(1/freq_sample)

    # Define axis
    ax = Axes(
          x_range=[-8, 8, 1],
          y_range=[0, 1.5, .75],
          tips=True,
          y_length = 4,
          x_length = 10,
          axis_config={
              "color" : BLACK,
              "font_size" : 24,
              "decimal_number_config" : {
                  "color" : "#000001",
                  "num_decimal_places" : 1,
                  "include_sign" : False,
              },
              "include_numbers": True}
      )

    # Calculate the fixed function
    f0 = func0(tmp)

    # start the slide function in 0 just with inversion
    t = ValueTracker(0)

    # define slide function
    def slide_func():
      f1 = func0(t.get_value()-tmp)
      line_graph = ax.plot_line_graph(
              x_values = tmp,
              y_values = f1,
              line_color='BLUE',
              add_vertex_dots=False,
              stroke_width = 4,
          )
      return line_graph

    graph = ax.plot_line_graph(
              x_values = tmp,
              y_values = f0,
              line_color='BLUE',
              add_vertex_dots=False,
              stroke_width = 4,
          )

    labels = ax.get_axis_labels(x_label=r"\tau", y_label=r"x(\tau)").set_color(BLACK)


    labels1 = ax.get_axis_labels(x_label=r"\tau", y_label=r"x(-\tau)").set_color(BLACK)
    # Define decimal to try print number changing, there's a bug in this part of the code
    decimal = DecimalNumber(number=0, num_decimal_places=1, include_sign=True)
    decimal.add_updater(lambda d: d.set_value(tmp.get_value())).to_corner(RIGHT).scale(.001)


    labels1_x = ax.get_x_axis_label(r"\tau").set_color(BLACK)
    labels1_y = always_redraw(lambda : ax.get_y_axis_label(r"x(" + r"{:.1f}".format(t.get_value()) + r"-\tau)").set_color(BLACK))

    labels2_y = always_redraw(lambda : ax.get_y_axis_label(r"x(\tau)\cdot x(" + r"{:.1f}".format(t.get_value()) + r"-\tau)").set_color(BLACK))

    graph1 = always_redraw(slide_func)

    # Create first animation with normal functions
    self.play(Create(ax))
    self.add(labels)
    self.wait()
    self.play(FadeIn(graph))
    self.wait(2)
    # shift the graph
    self.play(ReplacementTransform(labels, labels1))
    self.remove(graph)
    self.add(graph1)
    self.wait(2)

    self.play(ReplacementTransform(labels1, labels1_y))
    self.add(labels1_x)
    self.wait(2)
    self.play(t.animate.set_value(-2), run_time = .5, rate_func = linear)
    self.wait(2)
    self.play(t.animate.set_value(0), run_time = .5, rate_func = linear)
    self.wait(2)
    self.play(t.animate.set_value(2), run_time = .5, rate_func = linear)
    self.wait(2)

    # change colors and change position of labels
    self.play(t.animate.set_value(-4), run_time = .5, rate_func = linear)
    self.play(FadeOut(graph1))
    self.play(FadeOut(labels1_x))
    self.play(FadeOut(labels1_y))
    self.wait()

    # Start again

    self.add(labels1_x)
    self.add(labels2_y)
    self.wait()
    self.play(FadeIn(graph["line_graph"].set_color("#FF0000")))
    self.add(graph1)
    self.wait(2)
    self.play(t.animate.set_value(4), run_time = 2, rate_func = linear)
    self.wait()

Observa-se na animação acima, que a cada valor de $t$, tem-se um deslocamento da curva. A integral de convolução é calculada a cada deslocamento da curva "deslizante". Os valores diferentes de zero são obtidos na interseção dos sinais, sendo que a área vai do mínimo ao seu valor máximo e retorna para seu valor mínimo.

In [51]:
%%manim -qh -v WARNING ConvFunc

# define the function in parts
def func0(tmp):
  x = np.zeros_like(tmp)
  # assuming tmp is an array
  for i in range(len(tmp)):
    if(tmp[i] < -0.5 ):
      x[i] = 0
    elif(tmp[i] >= -0.5 and tmp[i] < 0.5):
      x[i] = 1
    else:
      x[i] = 0
  return x

class ConvFunc(Scene):
  def construct(self):
    self.camera.background_color = "#FFFFFF"

    # frequência de amostragem e vetor de tempo
    freq_sample = 500
    t_final = 5
    tmp = np.arange(-t_final*freq_sample, t_final*freq_sample+1)*(1/freq_sample)

    # Define axis
    ax = Axes(
          x_range=[-8, 8, 1],
          y_range=[0, 1.5, .75],
          tips=True,
          y_length = 3,
          x_length = 10,
          axis_config={
              "color" : BLACK,
              "font_size" : 24,
              "decimal_number_config" : {
                  "color" : "#000001",
                  "num_decimal_places" : 1,
                  "include_sign" : False,
              },
              "include_numbers": True}
      )
    # shift axis to fit two in the scene
    ax.shift(2*UP)
    ax2 = ax.copy().shift(3.5*DOWN)


    # Calculate the fixed function
    f0 = func0(tmp)
    # start the slide function in -4
    t = ValueTracker(-4)

    # define slide function
    def slide_func():
      f1 = func0(t.get_value()-tmp)
      line_graph = ax.plot_line_graph(
              x_values = tmp,
              y_values = f1,
              line_color='BLUE',
              add_vertex_dots=False,
              stroke_width = 4,
          )
      return line_graph
    # Based in examples frrom manim community
    self.curve_start = ax2.c2p(t.get_value(), 0, 0)
    self.curve = VGroup()
    self.curve.add(Line(self.curve_start, self.curve_start))
    def get_signal():
        f0 = func0(tmp)
        f1 = func0(t.get_value()-tmp)
        f3 = scipy.integrate.simps(f0*f1, tmp)

        last_line = self.curve[-1]
        #x = self.curve_start[0] + ax2.p2c(np.array([t.get_value(), 1, 0]))[0]
        #y = 1
        teste = ax2.c2p(t.get_value(), f3, 0)
        new_line = Line(last_line.get_end(), teste, color='RED')
        self.curve.add(new_line)

        return self.curve

    labels1_x = ax.get_x_axis_label(r"\tau").set_color(BLACK)
    labels1_y = always_redraw(lambda : ax.get_y_axis_label(r"x(\tau)\cdot x(" + r"{:.1f}".format(t.get_value()) + r"-\tau)").set_color(BLACK))

    labels2_x = ax2.get_x_axis_label(r"t").set_color(BLACK)
    labels2_y = always_redraw(lambda : ax2.get_y_axis_label(r"(f*g)(" + r"{:.1f}".format(t.get_value()) + r")").set_color(BLACK).shift(.8*DOWN))

    graph = ax.plot_line_graph(
              x_values = tmp,
              y_values = f0,
              line_color='BLACK',
              add_vertex_dots=False,
              stroke_width = 4,
          )

    graph1 = always_redraw(slide_func)
    graph2 = always_redraw(get_signal)

    self.play(Create(ax))
    self.wait()
    self.play(FadeIn(graph))
    self.add(labels1_x)
    self.wait()
    self.add(labels1_y)
    self.add(labels2_x)
    self.add(labels2_y)
    self.play(Create(ax2))
    self.play(FadeIn(graph1))
    self.play(FadeIn(graph2))
    self.wait(2)
    self.play(t.animate.set_value(4), run_time = 8, rate_func = linear)
    self.wait(2)